In [1]:
import glob
import pandas as pd
import numpy as np
import gc

In [2]:
present_path = glob.os.getcwd()
all_files = glob.glob(glob.os.path.join(glob.os.getcwd().split("\\智能风控与反欺诈第八章")[0], 'data\\*'))
all_files_names = [path.split("\\data\\")[-1].split(".csv")[0] for path in all_files]
all_files_dict = {}
for i in range(len(all_files_names)):
    all_files_dict[all_files_names[i]] = all_files[i]
all_files_dict

{'application_test': 'E:\\GitHub_Repositories\\Data-Analysis\\Kaggle_Projects\\Home Credit Default Risk\\data\\application_test.csv',
 'application_train': 'E:\\GitHub_Repositories\\Data-Analysis\\Kaggle_Projects\\Home Credit Default Risk\\data\\application_train.csv',
 'bureau': 'E:\\GitHub_Repositories\\Data-Analysis\\Kaggle_Projects\\Home Credit Default Risk\\data\\bureau.csv',
 'bureau_balance': 'E:\\GitHub_Repositories\\Data-Analysis\\Kaggle_Projects\\Home Credit Default Risk\\data\\bureau_balance.csv',
 'credit_card_balance': 'E:\\GitHub_Repositories\\Data-Analysis\\Kaggle_Projects\\Home Credit Default Risk\\data\\credit_card_balance.csv',
 'HomeCredit_columns_description': 'E:\\GitHub_Repositories\\Data-Analysis\\Kaggle_Projects\\Home Credit Default Risk\\data\\HomeCredit_columns_description.csv',
 'installments_payments': 'E:\\GitHub_Repositories\\Data-Analysis\\Kaggle_Projects\\Home Credit Default Risk\\data\\installments_payments.csv',
 'POS_CASH_balance': 'E:\\GitHub_Reposit

In [4]:
data_pre_application = pd.read_csv(all_files_dict["previous_application"])
[column_name for column_name in data_pre_application.columns if "DAYS" in column_name]

['DAYS_DECISION',
 'DAYS_FIRST_DRAWING',
 'DAYS_FIRST_DUE',
 'DAYS_LAST_DUE_1ST_VERSION',
 'DAYS_LAST_DUE',
 'DAYS_TERMINATION']

In [5]:
# 异常值替换为缺失
data_pre_application["DAYS_FIRST_DRAWING"].replace(365243, np.nan, inplace=True)
data_pre_application["DAYS_FIRST_DUE"].replace(365243, np.nan, inplace=True)
data_pre_application["DAYS_LAST_DUE_1ST_VERSION"].replace(365243, np.nan, inplace=True)
data_pre_application["DAYS_LAST_DUE"].replace(365243, np.nan, inplace=True)
data_pre_application["DAYS_TERMINATION"].replace(365243, np.nan, inplace=True)

In [7]:
# from sklearn.preprocessing import OneHotEncoder


# # 字符型特征离散化
# data_pre_application, new_columns_pre_application = one_hot_encoder(data_pre_application)
# data_pre_application.head()

In [9]:
# 衍生比例类特征
data_pre_application["APP_CREDIT_PERC"] = data_pre_application["AMT_APPLICATION"]/data_pre_application["AMT_CREDIT"]
data_pre_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,APP_CREDIT_PERC
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,12.0,middle,POS mobile with interest,NaN,-42.0,300.0,-42.0,-37.0,0.0,1.000000
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,36.0,low_action,Cash X-Sell: low,NaN,-134.0,916.0,NaN,NaN,1.0,0.893815
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,12.0,high,Cash X-Sell: high,NaN,-271.0,59.0,NaN,NaN,1.0,0.824511
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,12.0,middle,Cash X-Sell: middle,NaN,-482.0,-152.0,-182.0,-177.0,1.0,0.955840
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN,0.835282


In [11]:
# 衍生聚合类特征
num_aggregations = {
    "AMT_ANNUITY" : ["min", "max", "mean"], 
    "AMT_APPLICATION" : ["min", "max", "mean"],
    "AMT_CREDIT" : ["min", "max", "mean"], 
    "APP_CREDIT_PERC" : ["min", "max", "mean"], 
    "AMT_DOWN_PAYMENT" : ["min", "max", "mean"], 
    "AMT_GOODS_PRICE" : ["min", "max", "mean"], 
    "HOUR_APPR_PROCESS_START" : ["min", "max", "mean"], 
    "RATE_DOWN_PAYMENT" : ["min", "max", "mean"], 
    "DAYS_DECISION" : ["min", "max", "mean"], 
    "CNT_PAYMENT" : ["min", "max", "mean"]
}

# cat_aggregations = {}
# for cat in new columns_pre_application:
#     cat_aggregations[cat] = ["mean"]

In [18]:
# 历史申请特征
# prev_agg = data_pre_application.groupby("SK_ID_CURR").agg({**num_aggregations, **cat_aggregations})
prev_agg = data_pre_application.groupby("SK_ID_CURR").agg({**num_aggregations})
prev_agg.columns.tolist()

[('AMT_ANNUITY', 'min'),
 ('AMT_ANNUITY', 'max'),
 ('AMT_ANNUITY', 'mean'),
 ('AMT_APPLICATION', 'min'),
 ('AMT_APPLICATION', 'max'),
 ('AMT_APPLICATION', 'mean'),
 ('AMT_CREDIT', 'min'),
 ('AMT_CREDIT', 'max'),
 ('AMT_CREDIT', 'mean'),
 ('APP_CREDIT_PERC', 'min'),
 ('APP_CREDIT_PERC', 'max'),
 ('APP_CREDIT_PERC', 'mean'),
 ('AMT_DOWN_PAYMENT', 'min'),
 ('AMT_DOWN_PAYMENT', 'max'),
 ('AMT_DOWN_PAYMENT', 'mean'),
 ('AMT_GOODS_PRICE', 'min'),
 ('AMT_GOODS_PRICE', 'max'),
 ('AMT_GOODS_PRICE', 'mean'),
 ('HOUR_APPR_PROCESS_START', 'min'),
 ('HOUR_APPR_PROCESS_START', 'max'),
 ('HOUR_APPR_PROCESS_START', 'mean'),
 ('RATE_DOWN_PAYMENT', 'min'),
 ('RATE_DOWN_PAYMENT', 'max'),
 ('RATE_DOWN_PAYMENT', 'mean'),
 ('DAYS_DECISION', 'min'),
 ('DAYS_DECISION', 'max'),
 ('DAYS_DECISION', 'mean'),
 ('CNT_PAYMENT', 'min'),
 ('CNT_PAYMENT', 'max'),
 ('CNT_PAYMENT', 'mean')]

In [19]:
prev_agg.columns = pd.Index(["PREV_" + column[0] + '_' + column[1].upper() for column in prev_agg.columns.tolist()])
prev_agg

,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEAN,PREV_AMT_APPLICATION_MIN,PREV_AMT_APPLICATION_MAX,PREV_AMT_APPLICATION_MEAN,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,PREV_AMT_CREDIT_MEAN,PREV_APP_CREDIT_PERC_MIN,...,PREV_HOUR_APPR_PROCESS_START_MEAN,PREV_RATE_DOWN_PAYMENT_MIN,PREV_RATE_DOWN_PAYMENT_MAX,PREV_RATE_DOWN_PAYMENT_MEAN,PREV_DAYS_DECISION_MIN,PREV_DAYS_DECISION_MAX,PREV_DAYS_DECISION_MEAN,PREV_CNT_PAYMENT_MIN,PREV_CNT_PAYMENT_MAX,PREV_CNT_PAYMENT_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.000,3951.000000,24835.5,24835.5,24835.500,23787.0,23787.0,23787.00,1.044079,...,13.000000,0.104326,0.104326,0.104326,-1740,-1740,-1740.000,8.0,8.0,8.00
100002,9251.775,9251.775,9251.775000,179055.0,179055.0,179055.000,179055.0,179055.0,179055.00,1.000000,...,9.000000,0.000000,0.000000,0.000000,-606,-606,-606.000,24.0,24.0,24.00
100003,6737.310,98356.995,56553.990000,68809.5,900000.0,435436.500,68053.5,1035882.0,484191.00,0.868825,...,14.666667,0.000000,0.100061,0.050030,-2341,-746,-1305.000,6.0,12.0,10.00
100004,5357.250,5357.250,5357.250000,24282.0,24282.0,24282.000,20106.0,20106.0,20106.00,1.207699,...,5.000000,0.212008,0.212008,0.212008,-815,-815,-815.000,4.0,4.0,4.00
100005,4813.200,4813.200,4813.200000,0.0,44617.5,22308.750,0.0,40153.5,20076.75,1.111173,...,10.500000,0.108964,0.108964,0.108964,-757,-315,-536.000,12.0,12.0,12.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,6605.910,6605.910,6605.910000,40455.0,40455.0,40455.000,40455.0,40455.0,40455.00,1.000000,...,17.000000,0.000000,0.000000,0.000000,-273,-273,-273.000,8.0,8.0,8.00
456252,10074.465,10074.465,10074.465000,57595.5,57595.5,57595.500,56821.5,56821.5,56821.50,1.013622,...,10.000000,0.062443,0.062443,0.062443,-2497,-2497,-2497.000,6.0,6.0,6.00
456253,3973.095,5567.715,4770.405000,19413.0,28912.5,24162.750,13945.5,27306.0,20625.75,1.058833,...,11.500000,0.104349,0.324284,0.214316,-2851,-1909,-2380.000,4.0,6.0,5.00


In [22]:
data_pre_application.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL',
       'APP_CREDIT_PERC'],
      dtype='object')

In [23]:
# # 历史申请通过特征
# approved = data_pre_application[data_pre_application['NAME_CONTRACT_STATUS_Approved'] == 1]
# approved_agg = approved.groupby("SK_ID_CURR").agg(num_aggregations)
# approved_agg.columns = pd.Index(["APPROVED_" + column[0] + '_' + column[1].upper() for column in approved_agg.columns.tolist()])
# prev_agg = prev_agg.join(approved_agg, how='left', on="SK_ID_CURR")
# del approved, approved_agg

In [24]:
# # 历史申请拒绝特征
# refused = data_pre_application[data_pre_application['NAME_CONTRACT_STATUS_Refused'] == 1]
# refused_agg = refused.groupby("SK_ID_CURR").agg(num_aggregations)
# refused_agg.columns = pd.Index(["REFUSED_" + column[0] + '_' + column[1].upper() for column in approved_agg.columns.tolist()])
# prev_agg = prev_agg.join(refused_agg, how='left', on="SK_ID_CURR")
# del refused, refused_agg, data_pre_application

gc.collect()

24139